# 잡플래닛 스크레이핑
- IT기업 리스트 
- 기업별 리뷰데이터 

# 라이브러리

In [ ]:
# dataframe 관련 라이브러리
import pandas as pd
import numpy as np

# 스크레이핑 관련 라이브러리
from bs4 import BeautifulSoup 
import requests
from concurrent.futures import ThreadPoolExecutor #멀티스레드
import math #로딩 페이지 수 계산 용

import os

# 데이터 수집

## 기업 리스트

In [ ]:
# 잡플래닛 로그인 관련 정보
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'', 'password':'', 'remember_me':'true'}} # 본인의 계정정보 입력

#회사목록 url-list
URLs = [f'https://www.jobplanet.co.kr/companies?industry_id=700&page={page_num}' for page_num in range(1,442)]

def get_url(url):
    client = requests.session()
    login_response = client.post(url, json = login_data, headers = headers)
    return requests.get(url)

with ThreadPoolExecutor(max_workers=10) as pool:
    response_list = list(pool.map(get_url, URLs))

cnt = 0
# 데이터 프레임 형성
df_company_list = pd.DataFrame(columns=("회사명", "회사코드", "업종", "본사위치", "리뷰수", "평균별점", "평균연봉"))

for response in response_list:
    soup = BeautifulSoup(response.text, 'html.parser')
    sites = soup.find_all('div', {'class' :'ty3_wrap'})

    for site in sites:
        # select는 리스트로 들어오기 때문에 인덱싱 해야함.
        name = site.select('dt > a')[0].get_text()
        code = site.button['data-company_id']        
        category = site.select('dd > span')[0].get_text()
        location = site.select('dd > span')[2].get_text()
        num_reviews = site.select('dl > dt')[1].get_text().rstrip('개의 리뷰').replace(',','')
        avg_star = site.select('span.gfvalue')[0].get_text()
        avg_salary = site.select('a > strong')[0].get_text().replace(',','')

        df_company_list.loc[cnt] = [name, code, category, location, num_reviews, avg_star, avg_salary]
        cnt += 1

- review 수 30개 미만 데이터는 제거
    - 중심극한 정리에서 정규분포 가정이 성립하기위한 최소 표본의 크기는 30개 

In [ ]:
#30개의 마지막은 index 1200번
df_company_list[df_company_list.리뷰수 == '30']

# 리뷰 30개 미만 기업 데이터 제외
df_company_list = df_company_list[:1201]

In [ ]:
# 기업 리스트 csv파일 내보내기 
# df_company_list.to_csv('./input/company_list.csv')

## 기업별 리뷰데이터

In [ ]:
# 기업 리스트 csv 불러오기
df_company_list = pd.read_csv('./input/df_company_list.csv', index_col=0)
df_company_list.head()

- 위에서 수집한 기업 리스트를 이용하여 url을 변경하며 스크레이핑 진행

In [ ]:
# 스크레이핑시 url에 (주), (유), (재) 가 없는 회사명이 들어가므로
# 리스트에 있는 회사명의 (주), (유), (재) 를 제외한 회사명 컬럼 생성
df_company_list['회사_new'] = df_company_list['회사명'].str.replace(r'\([가-힣]\)', '')
df_company_list.head(3)

In [ ]:
for code, name, reviews in zip(df_company_list['회사코드'],df_company_list['회사_new'], df_company_list['리뷰수']):
    print(code,name,reviews)

In [ ]:
# 잡플래닛 로그인 
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'', 'password':'', 'remember_me':'true'}} # 본인의 계정정보 입력

client = requests.session()
URL = 'https://www.jobplanet.co.kr/users/sign_in'
login_response = client.post(URL, json = login_data, headers = headers)

cnt = 0
순번 = 0

for code, name, num in zip(company_code_list, company_list, review_nums):    
    
    # 회사별 max page 생성      
    max_page_num = math.ceil(int(num) / 5)
    
    진행률 = 1       
    
    print('현재 진행중인 기업명 :',순번,name)
    for page_num in range(1,max_page_num+1):
        URL = f'https://www.jobplanet.co.kr/companies/{code}/reviews/{name}?page={page_num}'
        index = client.get(URL)
        # soup = BeautifulSoup(index.text, 'html.parser')  -> <br> 태그 때문에 여러줄 데이터 받는게 어려움
        soup = BeautifulSoup(index.text, 'html5lib') # br 태그로 여러줄 데이터가 잘려서 나오는거 방지
        sites = soup.find_all('section', {'class' :'content_ty4 video_ad_content'})
        
        for site in sites:
            # 회사명, 코드
            회사명 = name
            회사코드 = code
            
            # 직군, 전/현직원 구분, 근무지역(본사 아니면 불만 있는 경우 있음), 작성년월
            # 근무지역 결측치 발생시 예외처리
            
            if site.select('div > span')[0].get_text() == '':
                직군 = np.NaN
                전현직 = '현직원'
                근무지역 = np.NaN
                작성월 = np.NaN
            
            elif site.select('div > span')[1].get_text() == '|':
                직군 = np.NaN
                전현직 = '현직원'
                근무지역 = np.NaN
                작성월 = site.select('div > span')[2].get_text()

            
            elif site.select('div > span')[5].get_text() == '|':
                직군 = site.select('div > span')[1].get_text()
                전현직 = site.select('div > span')[3].get_text().strip()
                근무지역 = np.NaN          
                작성월 = site.select('div > span')[6].get_text()
                        
            
            else:
                직군 = site.select('div > span')[1].get_text()
                전현직 = site.select('div > span')[3].get_text().strip()
                근무지역 = site.select('div > span')[5].get_text()            
                
                try:
                    작성월 = site.select('div > span')[7].get_text()
                except:
                    작성원 = np.NaN



            #별점, 승진 기회 및 가능성, 복지 및 급여, 업무와 삶의 균형, 사내문화, 경영진 (점수)    
            별점 = int( site.select('div > dl > dd > div > div')[0]['style'][6:].replace('%;','') ) // 20
            승진 = int( site.select('div > dl > dd > div > div')[1]['style'][6:].replace('%;','') ) // 20
            복지 = int( site.select('div > dl > dd > div > div')[2]['style'][6:].replace('%;','') ) // 20
            업무 = int( site.select('div > dl > dd > div > div')[3]['style'][6:].replace('%;','') ) // 20
            문화 = int( site.select('div > dl > dd > div > div')[4]['style'][6:].replace('%;','') ) // 20
            경영진점수 = int( site.select('div > dl > dd > div > div')[5]['style'][6:].replace('%;','') ) // 20
            
            # print(회사명, 회사코드, 직군, 전현직, 근무지역, 작성월, 별점, 승진, 복지, 업무, 문화, 경영진)
            #요약
            요약 = site.select('div > h2')[0].get_text().strip('\nBEST\n " "\n ')
            # print(요약)
            장점 = site.select('div > div > div > dl > dd > span')[0].get_text()
            단점 = site.select('div > div > div > dl > dd > span')[1].get_text()
            경영진 = site.select('div > div > div > dl > dd > span')[2].get_text()            
            try:           
                일년후 = site.select('div > p > strong')[0].get_text()
            except:
                일년후 = np.NaN
            try:
                추천여부 = site.select('p > em')[0].get_text()
            except:
                추천여부 = '비추천'
            게시글추천수 = int(site.select('a > span')[1].get_text())

            df_reviews.loc[cnt] = [회사명, 회사코드, 직군, 전현직, 근무지역, 작성월, 별점, 승진, 복지,
                                   업무, 문화, 경영진점수, 요약, 장점, 단점, 경영진, 일년후, 추천여부, 게시글추천수
                                   ]
            cnt +=1
            
            
            if 진행률 == reviews :
                print(진행률,'/',reviews,'개 완료')
                display(df_reviews.tail(1))
                진행률+=1
    순번 += 1

# 내보내기
df_reviews.to_csv('./input/df_reviews.csv')